In [64]:
from Bio import SeqIO
from pathlib import Path
from Bio.SeqRecord import SeqRecord
import pandas as pd
import re

identifier = r"CYP[\d+][A-Z+][\d+]"
pattern_family = re.compile(r"(?<=CYP)[\d]+")
pattern_id = re.compile(r"(?<=CYP)[\d]+[A-Z]+[\d]+")

test = "CYP44LK4"
print(re.search(pattern_family, test).group())
print(re.search(pattern_id, test).group())


def match_or_none(pattern, s):
    try:
        out = re.search(pattern, s).group()
    except:
        out = None
    return out

44
44LK4


In [ ]:
data_dir = Path("/hits/fast/cme/bodynems/data")
seqs: list[SeqRecord] = list(
    SeqIO.parse(data_dir / "backup" / "arthropod.fasta", format="fasta")
)
print(len(seqs))

stats = []
for seq in seqs:
    stats.append([seq.id, len(seq)])

stats_df = pd.DataFrame(stats, columns=["id_full", "length"])


stats_df["id"] = stats_df["id_full"].map(lambda s: match_or_none(pattern_id, s))
stats_df["id"].value_counts()
stats_df["family"] = 

# stats_df["subfamily"] = stats_df["id"].map(
#     lambda s: match_or_none(pattern_subfamily, s)
# )
# stats_df["subfamily"].value_counts()

11118


id
4G15     153
314A1    122
4G1      114
302A1    114
315A1     92
        ... 
4DM3       1
4DM1       1
4DM2       1
4DL1       1
4W7        1
Name: count, Length: 8511, dtype: int64